In [23]:
from langchain_openai import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler

llm = ChatOpenAI(
    temperature=0.1,
    model="gpt-4o-mini",
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

In [24]:
from langchain_community.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain.vectorstores import FAISS


def embed_file():
    cache_dir = LocalFileStore("./.cache/")
    loader = UnstructuredFileLoader("./files/document.txt")
    splitter = CharacterTextSplitter.from_tiktoken_encoder(
        separator="\n",
        chunk_size=600,
        chunk_overlap=50,
    )
    docs = loader.load_and_split(text_splitter=splitter)
    embedder = OpenAIEmbeddings()
    cache_embedder = CacheBackedEmbeddings.from_bytes_store(embedder, cache_dir)
    vectorStore = FAISS.from_documents(docs, cache_embedder)
    retriver = vectorStore.as_retriever()

    return retriver

In [25]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    return_messages=True,
    memory_key="history"
)

def save_memory(input, output):
    memory.save_context({"input": input}, {"output" :  output})

def load_memory(_):
    return memory.load_memory_variables({})["history"]

In [26]:
def format_doc(document):
    return "\n\n".join(doc.page_content for doc in document)

In [27]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a helpful assistant. Answer questions using only the following context. and You remember conversations with human.
            DON'T make it up.
            --------
            {context}
            """,
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

retriver = embed_file()

chain = (
    {
        "context": retriver | RunnableLambda(format_doc),
        "history": RunnableLambda(load_memory),
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
)


def invoke_chain(question):
    result = chain.invoke(question).content
    save_memory(question, result)

libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.


In [28]:
invoke_chain("Is Aaronson guilty?")

According to the context provided, Winston believes that Jones, Aaronson, and Rutherford are guilty of the crimes they are charged with, and he has convinced himself that he had never seen the photograph that disproved their guilt, which he had invented. Therefore, within the framework of Winston's thoughts, Aaronson is considered guilty.

In [29]:
invoke_chain("What message did he write in the table?")

The name of the person you just asked about is Aaronson.

In [30]:
invoke_chain("Who is Julia?")

Julia is a character who is Winston's love interest. In the context provided, she is someone with whom Winston shares a deep emotional connection, and he experiences overwhelming feelings for her, especially in moments of distress. Julia represents a form of rebellion against the Party, and Winston feels a strong desire to protect her.

In [31]:
load_memory(1)

[HumanMessage(content='Is Aaronson guilty?'),
 AIMessage(content="According to the context provided, Winston believes that Jones, Aaronson, and Rutherford are guilty of the crimes they are charged with, and he has convinced himself that he had never seen the photograph that disproved their guilt, which he had invented. Therefore, within the framework of Winston's thoughts, Aaronson is considered guilty."),
 HumanMessage(content="Who's the name of the person I just asked?"),
 AIMessage(content='The name of the person you just asked about is Aaronson.'),
 HumanMessage(content='Who is Julia?'),
 AIMessage(content="Julia is a character who is Winston's love interest. In the context provided, she is someone with whom Winston shares a deep emotional connection, and he experiences overwhelming feelings for her, especially in moments of distress. Julia represents a form of rebellion against the Party, and Winston feels a strong desire to protect her.")]